In [19]:
!git clone https://github.com/henryNTUEE/tf-tutorial.git

Cloning into 'tf-tutorial'...
remote: Counting objects: 80, done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 80 (delta 16), reused 76 (delta 14), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [20]:
%cd tf-tutorial/tensorflow_lyrics_bot/

/gpfs/global_fs01/sym_shared/YPProdSpark/user/s940-fbcc213783342d-a0c48b40f6b0/notebook/work/tf-tutorial/tensorflow_lyrics_bot


In [21]:
# The code was removed by DSX for sharing.

In [22]:
print (data.getvalue().split('\n')[0])

下課 花路 米   第一 集   探索 蟋蟀 之旅


In [23]:
with open('works/subtitle/data/train/chat.txt','w') as w:
    for line in data:
        w.write(line + '\n')

In [24]:
%cd works/subtitle/data/train/

/gpfs/global_fs01/sym_shared/YPProdSpark/user/s940-fbcc213783342d-a0c48b40f6b0/notebook/work/tf-tutorial/tensorflow_lyrics_bot/works/subtitle/data/train


In [27]:
!gzip chat.txt

In [28]:
%cd ../../../../

/gpfs/global_fs01/sym_shared/YPProdSpark/user/s940-fbcc213783342d-a0c48b40f6b0/notebook/work/tf-tutorial/tensorflow_lyrics_bot/works


In [ ]:
# %load main.py
import os, sys, argparse

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

import tensorflow as tf

from lib.config import params_setup
from lib.train import train
from lib.predict import predict
from lib.chat import chat
# from lib.mert import mert


def main(_):
    args = params_setup()
    print("[args]: ", args)
    if args.mode == 'train':
      train(args)
    elif args.mode == 'test':
      predict(args)
    elif args.mode == 'chat':
      chat(args)
    # elif args.mode == 'mert':
    #   mert(args)


if __name__ == "__main__":
    tf.app.run()

In [ ]:
# %load lib/train.py
import sys, os, math, time, argparse, shutil, gzip
import numpy as np
import tensorflow as tf

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from six.moves import xrange  # pylint: disable=redefined-builtin
from datetime import datetime
from lib import seq2seq_model_utils, data_utils


def setup_workpath(workspace):
  for p in ['data', 'nn_models', 'results']:
    wp = "%s/%s" % (workspace, p)
    if not os.path.exists(wp): os.mkdir(wp)

  data_dir = "%s/data" % (workspace)
  # training data
  if not os.path.exists("%s/chat.in" % data_dir):
    n = 0
    f_zip   = gzip.open("%s/train/chat.txt.gz" % data_dir, 'rt')
    f_train = open("%s/chat.in" % data_dir, 'w')
    f_dev   = open("%s/chat_test.in" % data_dir, 'w')
    for line in f_zip:
      f_train.write(line)
      if n < 10000: 
        f_dev.write(line)
        n += 1


def train(args):
    print("[%s] Preparing dialog data in %s" % (args.model_name, args.data_dir))
    setup_workpath(workspace=args.workspace)
    train_data, dev_data, _ = data_utils.prepare_dialog_data(args.data_dir, args.vocab_size)

    if args.reinforce_learn:
      args.batch_size = 1  # We decode one sentence at a time.

    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=args.gpu_usage)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:

        # Create model.
        print("Creating %d layers of %d units." % (args.num_layers, args.size))
        model = seq2seq_model_utils.create_model(sess, args, forward_only=False)

        # Read data into buckets and compute their sizes.
        print("Reading development and training data (limit: %d)." % args.max_train_data_size)
        dev_set = data_utils.read_data(dev_data, args.buckets, reversed=args.rev_model)
        train_set = data_utils.read_data(train_data, args.buckets, args.max_train_data_size, reversed=args.rev_model)
        train_bucket_sizes = [len(train_set[b]) for b in xrange(len(args.buckets))]
        train_total_size = float(sum(train_bucket_sizes))

        # A bucket scale is a list of increasing numbers from 0 to 1 that we'll use
        # to select a bucket. Length of [scale[i], scale[i+1]] is proportional to
        # the size if i-th training bucket, as used later.
        train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                               for i in xrange(len(train_bucket_sizes))]

        # This is the training loop.
        step_time, loss = 0.0, 0.0
        current_step = 0
        previous_losses = []

        # Load vocabularies.
        vocab_path = os.path.join(args.data_dir, "vocab%d.in" % args.vocab_size)
        vocab, rev_vocab = data_utils.initialize_vocabulary(vocab_path)

        while True:
          # Choose a bucket according to data distribution. We pick a random number
          # in [0, 1] and use the corresponding interval in train_buckets_scale.
          random_number_01 = np.random.random_sample()
          bucket_id = min([i for i in xrange(len(train_buckets_scale))
                           if train_buckets_scale[i] > random_number_01])

          # Get a batch and make a step.
          start_time = time.time()
          encoder_inputs, decoder_inputs, target_weights = model.get_batch(
              train_set, bucket_id)

          # print("[shape]", np.shape(encoder_inputs), np.shape(decoder_inputs), np.shape(target_weights))
          if args.reinforce_learn:
            _, step_loss, _ = model.step_rf(args, sess, encoder_inputs, decoder_inputs,
                                         target_weights, bucket_id, rev_vocab=rev_vocab)
          else:
            _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                         target_weights, bucket_id, forward_only=False, force_dec_input=True)

          step_time += (time.time() - start_time) / args.steps_per_checkpoint
          loss += step_loss / args.steps_per_checkpoint
          current_step += 1

          # Once in a while, we save checkpoint, print statistics, and run evals.
          if (current_step % args.steps_per_checkpoint == 0) and (not args.reinforce_learn):
            # Print statistics for the previous epoch.
            perplexity = math.exp(loss) if loss < 300 else float('inf')
            print ("global step %d learning rate %.4f step-time %.2f perplexity %.2f @ %s" %
                   (model.global_step.eval(), model.learning_rate.eval(), step_time, perplexity, datetime.now()))

            # Decrease learning rate if no improvement was seen over last 3 times.
            if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
              sess.run(model.learning_rate_decay_op)

            previous_losses.append(loss)

            # # Save checkpoint and zero timer and loss.
            checkpoint_path = os.path.join(args.model_dir, "model.ckpt")
            model.saver.save(sess, checkpoint_path, global_step=model.global_step)
            step_time, loss = 0.0, 0.0

            # Run evals on development set and print their perplexity.
            for bucket_id in xrange(len(args.buckets)):
              encoder_inputs, decoder_inputs, target_weights = model.get_batch(dev_set, bucket_id)
              _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, 
                                          target_weights, bucket_id, forward_only=True, force_dec_input=False)

              eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
              print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))

            sys.stdout.flush()


In [33]:
%run main.py --mode train --model subtitle

[args]:  Namespace(antilm=0, batch_size=64, beam_size=1, buckets=[(5, 10), (10, 15), (20, 25)], data_dir='works/subtitle/data', en_tfboard=0, gpu_usage=1.0, learning_rate=0.5, learning_rate_decay_factor=0.99, max_gradient_norm=5.0, max_train_data_size=0, mert_dataset_path='works/subtitle/data/test/mert_set.txt', mode='train', model_dir='works/subtitle/nn_models', model_name='subtitle', n_bonus=0, num_layers=4, reinforce_learn=0, results_dir='works/subtitle/results', rev_model=0, scope_name='subtitle', size=256, steps_per_checkpoint=500, test_dataset_path='works/subtitle/data/test/test_set.txt', tf_board_dir='works/subtitle/tf_board', vocab_size=150000, work_root='works', workspace='works/subtitle')
[subtitle] Preparing dialog data in works/subtitle/data
Creating vocabulary works/subtitle/data/vocab150000.in from data works/subtitle/data/chat.in
Tokenizing data in works/subtitle/data/chat.in
Tokenizing data in works/subtitle/data/chat_test.in
Creating 4 layers of 256 units.
Created mode

KeyboardInterrupt: 

In [34]:
from io import BytesIO  
import requests  
import json  
import pandas as pd
import tarfile
import codecs
import gzip


def put_file(local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    my_data = gzip.open(local_file_name)

    #my_data = open(local_file_name,'r')
    #my_data = codecs.open(local_file_name, "r",encoding='utf-8', errors='ignore')
    #f = open(local_file_name,'r')
    #my_data = f.read()
    
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_fb8e5938c53df784b12cd53aadbd3c904a812670','domain': {'id': 'a6ca2738c3ca49b3be81f963f7c0c794'},
            'password': 'D-lp89724?nTlj^g'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', 'Chatbotloadfile', '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'Accept-Encoding': None}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print(resp2)

In [36]:
!tar -zcvf subtitle.tar.gz works/subtitle/

works/subtitle/
works/subtitle/results/
works/subtitle/nn_models/
works/subtitle/nn_models/model.ckpt-500.meta
works/subtitle/nn_models/model.ckpt-500.index
works/subtitle/nn_models/checkpoint
works/subtitle/nn_models/model.ckpt-500.data-00000-of-00001
works/subtitle/data/
works/subtitle/data/chat_test.in
works/subtitle/data/train/
works/subtitle/data/train/chat.txt.gz
works/subtitle/data/chat.in
works/subtitle/data/chat.ids150000.in
works/subtitle/data/test/
works/subtitle/data/test/test_set.txt
works/subtitle/data/vocab150000.in
works/subtitle/data/chat_test.ids150000.in


In [ ]:
put_file('subtitle.tar.gz')